# MNISTデータセットにおける手書き数字認識
MINSTは機械学習分野で最も有名なデータセットの1つ
## データセットについて
このデータセットは、０〜９の手書きの画像があり、訓練画像が60,000枚、テスト画像が10,000枚用意されている。

### 読み込み〜

In [17]:
import sys, os
import numpy as np
import pickle

In [3]:
import sys, os
os.chdir('/users/isonoyuma/deep-learning-from-scratch/ch03')#カレントディレクトリ変更モジュール os.chdir
sys.path.append(os.pardir)#親ディレクトリのファイルをインポートするための設定
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [6]:
import sys, os
import numpy as np
os.chdir('/users/isonoyuma/deep-learning-from-scratch/ch03')#カレントディレクトリ変更モジュール os.chdir
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) #PIL用データオブジェクトに変換
    pil_img.show()
    
(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28,28)#flatten = Trueでダウンロードしたため、一時配列になっているのを元に戻す
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


## 実際にやってみる

#### sigmoid function

In [12]:
def sigmoid(x):#シグモイド関数(一般的な活性化関数)
    return 1/(1 + np.exp(-x))

#### softmax function

In [13]:
def softmax(a):#ソフトマックス関数(クラス分類問題における出力層の活性化関数)
    c = np.max(a)
    exp_a = np.exp(a-c)#オーバーフロー対策
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [25]:
def get_data():#学習データ及びテストデータの読み込み関数
    #今回は学習済みのデータがあったので、教師データは不必要なため、テストデータのみを返す。
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize = False)
    return x_test,t_test

def init_network():#ネットワーク初期化関数
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    return network

def predict(network, x):#ニューラルネットワークの開始関数
    W1,W2,W3 = network['W1'],network['W2'],network['W3']
    b1,b2,b3 = network['b1'],network['b2'],network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2,W3) + b3
    y = softmax(a3)
    
    return y

In [26]:
x,t = get_data()#テストデータ読み込み
network = init_network()#ネットワーク初期化

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
        
print("Accuracy:"+str(float(accuracy_cnt) / len(x)))

/Users/isonoyuma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Accuracy:0.9207


### バッチ処理による実装例

In [27]:
x,t = get_data()#テストデータ読み込み
network = init_network()#ネットワーク初期化

batch_size = 100 #バッチの数
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i + batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
        
print("Accuracy:"+str(float(accuracy_cnt) / len(x)))

Accuracy:0.9207


/Users/isonoyuma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
